In [2]:
import pyquasar as pq
import matplotlib.pyplot as plt
import numpy as np
import pyvista as pv

np.set_printoptions(precision=3, suppress=True)
%config InlineBackend.figure_format='retina'
%load_ext line_profiler

In [3]:
plt.rcParams.update(
  {
    "figure.figsize": (6, 6),
    "axes.facecolor": "none",
    "font.size": 10,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
  }
)

In [4]:
mesh = pq.Mesh.load("tetra.geo", refine_k=5)
# mesh = pq.Mesh.load("cube.geo", refine_k=0)
mesh

<Mesh object summary 
	Numeration: global
	Domains: [<MeshDomain object summary
	Material: steel
	Total elements number: 32768
	Element type: Tetrahedron 4; Count: 32768
	Boundary type: dirichlet; Tag: 1; Element type: Triangle 3; Count: 1024.
	Boundary type: neumann; Tag: 2; Element type: Triangle 3; Count: 1024.
	Boundary type: neumann; Tag: 3; Element type: Triangle 3; Count: 1024.
	Boundary type: neumann; Tag: 4; Element type: Triangle 3; Count: 1024.
>]>

In [5]:
def u(p, n):
  # return p[..., 0] ** 2 - p[..., 1] ** 2
  return 2 * p[..., 0] + 2 * p[..., 1] + 2 * p[..., 2]
  return 2 * p[..., 0] + 3 * p[..., 1] + 5 * p[..., 2] - 4


def flow(p, n):
  # return 2 * p[..., 0] * n[..., 0] - 2 * p[..., 1] * n[..., 1]
  return 2 * n[..., 0] + 2 * n[..., 1] + 2 * n[..., 2]
  return 2 * n[..., 0] + 3 * n[..., 1] + 5 * n[..., 2]

In [6]:
materials = {
  "dirichlet": u,
  "steel": {"neumann": flow, "steel": 0},
  "air": {"neumann": flow, "air": 0},
}

domains = [pq.FemDomain(domain) for domain in mesh.domains]
problem = pq.FemProblem(domains)
problem.assembly(materials)
# problem.reassembly_load(materials)
problem.add_skeleton_projection(u, ["dirichlet"])
sol = problem.solve()
rel_err = np.linalg.norm(sol - u(mesh.domains[0].vertices, 0)) / np.linalg.norm(u(mesh.domains[0].vertices, 0))
print(f"Relative error: {rel_err:.2e}")

Relative error: 4.74e-13


In [11]:
problem.factorize()

In [12]:
np.linalg.norm(problem.solve() - u(mesh.domains[0].vertices, 0)) / np.linalg.norm(u(mesh.domains[0].vertices, 0))

3.9748288436905493e-13

In [7]:
# points = mesh.domains[0].vertices
# cells = np.array([np.insert(element, 0, 4) for element in mesh.domains[0].elements[0].node_tags])
# # cells1 = np.array([np.insert(element, 0, 4) for element in mesh.domains[0].elements[0].node_tags])
# # cells2 = np.array([np.insert(element, 0, 4) for element in mesh.domains[1].elements[0].node_tags])
# # cells = np.concatenate([cells1, cells2])
# celltypes = [pv.CellType.TETRA] * cells.shape[0]

# pv.set_jupyter_backend("trame")
# # pv.set_jupyter_backend("static")

# grid = pv.UnstructuredGrid()
# grid = pv.UnstructuredGrid(cells, celltypes, points)
# grid.point_data["u"] = sol
# grid.plot(show_edges=True)

In [15]:
points = np.array(
  [
    [0, 0, 0],
    [0.1, 0.1, 0.1],
    [0.5, 0, 0],
    [0.5, 0, 0.5],
    [0, 1.0, 0],
    [1.0, 0.0, 0.0],
    [0, 0, 1]
  ]
)

In [7]:
proj_grad = problem.project_grad_into(mesh.domains[0].vertices[:100])
true_grad_x = proj_grad[0] @ problem.solve()
true_grad_y = proj_grad[1] @ problem.solve()
true_grad_z = proj_grad[2] @ problem.solve()

In [22]:
def func(flow, proj_grad, true_grad_x, true_grad_y, true_grad_z):
  problem.reassembly_load(
    {
      "dirichlet": u,
      "steel": {"neumann": flow, "steel": 0},
      # "air": {"neumann": flow, "air": 0},
    }
  )
  sol = problem.solve()
  grad_x = proj_grad[0] @ sol
  grad_y = proj_grad[1] @ sol
  grad_z = proj_grad[2] @ sol
  return np.sum((grad_x - true_grad_x) ** 2) + np.sum((grad_y - true_grad_y) ** 2) + np.sum((grad_z - true_grad_z) ** 2)

In [23]:
func(flow, proj_grad, true_grad_x, true_grad_y, true_grad_z)

0.0

$A * x = f$

$F * y = f$

$P_x * x = grad_x$

$P_x * A^{-1} * F * y = grad_x$

$M_x = P_x * A^{-1} * F$ - need block multiplication

$M = (M_x, M_y, M_z)$

$grad = (grad_x, grad_y, grad_z)$

$M * y = grad$

A shape: (dof, dof)

Px, Py, Pz shape: (p, dof)

M shape: (3p, dof)

F shape: (dof, g)

$A^{-1} * F = X => F = AX$ - SLAE

for f in F:
  x = spsolve(A, f) # f.shape: (dof, 1)
  M[i] = P_x @ x
